In [0]:
import gc
import json
import random
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sn
import pandas as pd
import numpy as np
import keras.backend as K

from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report
from keras.utils import np_utils
from keras.models import Model, load_model
from keras.layers import Input, Conv2D, MaxPooling2D, GlobalMaxPooling2D, Dense, Flatten, UpSampling2D, BatchNormalization, Dropout, GaussianNoise, Flatten
from keras.regularizers import l1,l2
from keras.optimizers import SGD, Adam, RMSprop
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

In [0]:
def set_random_seeds(seed=42):
    '''Sets seed in python random number generator, numpy and tensorflow
    

    Parameters:
    
    seed(int): seed number 

    '''
    random.seed(seed)
    np.random.seed(seed)
    tf.compat.v1.set_random_seed(seed)

In [0]:
def load_saved_model(filename):
    '''Returns a saved model.
    

    Parameters:
    
    filename(string): name of the saved model to load 

    '''
    return load_model('models/'+ filename +'.h5')

In [0]:
def load_history(filename):
    '''Returns an instance of History 

    Parameters: 
    
    filename(string): name of the json file containing a model history
    without the extension

    
    Returns:

    config(dict): dictionary with configuration

    '''
    history=None
    filepath= 'history/'+ filename +'.json'
    with open(filepath,'r') as fp:
        history= json.load(fp)

    return history
    

In [0]:
def save_history(history, filename):
    '''Saves a dictionary with a model configuration in the working
    directory as a json file
    

    Parameters:
    
    config(dict): dictionary with model configuration

    filepath(string): filepath where the configuration 
    should be saved

    '''
    filepath= 'history/' + filename + '.json'    
    with open(filepath,'w') as fp:
        json.dump(history,fp)

In [0]:
def load_config(filepath):
    '''Returns a dictionary with a model configuration loaded from working
    directory.
    

    Parameters: 
    
    filepath(string): path to the configuration json file

    
    Returns:

    config(dict): dictionary with configuration

    '''
    config= None
    with open(filepath, 'r') as fp:
        config= json.load(fp)
    return config 

In [0]:
def save_config(config, filepath):
    '''Saves a dictionary with a model configuration in the working
    directory as a json file
    

    Parameters:
    
    config(dict): dictionary with model configuration

    filepath(string): filepath where the configuration 
    should be saved

    '''    
    with open(filepath,'w') as fp:
        json.dump(config,fp)

In [0]:
def plot_confusion_matrix(cm, classes, normalize=True, figsize=(9,5)):
    ''' Plots confusion matrix

    Parameters:

    cm (confusion matrix 2darray): confusion matrix array

    classes(list): list containing unique class names used in the classification

    normalize(boolean): whether to normalize confusion matrix 

    figsize(tuple): tuple with width and height of confusion matrix plot

    '''
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        title='Normalized confusion matrix'
    else:
        title='Confusion matrix'

    fmt= '.2f' if normalize else 'd'
    df_cm = pd.DataFrame(cm, index=classes, columns=classes)
    fig = plt.figure(figsize=figsize)
    heatmap = sn.heatmap(df_cm, annot=True, fmt=fmt, cmap='Greys')
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), 
                                 rotation=0, 
                                 ha='right')
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), 
                                 rotation=45, 
                                 ha='right')
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.title(title)
    plt.tight_layout()
    plt.show()

In [0]:
def assert_kernel_regularizer(config):
    ''' Returns a Regularizer object or None based on config

    
    Parameters:

    config(dict): dictionary with model configuration

    
    Returns:

    depending on config, returns keras Regularizer instance l1 or l2, or None

    '''
    if config['kernel_regularizer']:
        if config['kernel_regularizer_type']=='l1':
            return l1(config['kernel_regularizer_value'])
        elif (config['kernel_regularizer_type']=='l2'):
            return l2(config['kernel_regularizer_value'])
        else:
            return l2()
    else:
        return None

In [0]:
def assert_activity_regularizer(config):
    ''' Returns a Regularizer instance or None based on config
    
    
    Parameters:

    config(dict): dictionary with model configuration

    
    Returns:

    depending on config, returns keras Regularizer instance l1 or l2, or None

    '''
    if config['activity_regularizer']:
        if config['activity_regularizer_type']=='l1':
            return l1(config['activity_regularizer_value'])
        elif config['activity_regularizer_type']=='l2':
            return l2(config['activity_regularizer_value'])
        else:
            return None
    else:
        return None

In [0]:
def assert_optimizer(config):
    ''' Returns an optimizer object based on config.

    
    Parameters:

    config(dict): dictionary with model configuration

    
    Returns:

    depending on config, returns keras Optimizer instance Adam, SGD, or RMSprop

    '''
    if config['optimizer']=='adam':
        return Adam(lr=config['lr'])
    elif config['optimizer']=='sgd':
        return SGD(lr=config['lr'],momentum=True)
    elif config['optimizer']=='rmsprop':
        return RMSprop(lr=config['lr'])
    else:
        return SGD

In [0]:
def get_class_weights(y,config):
    ''' Returns a dictionary of class weights.

    
    Parameters:

    y(2d array): one-hot encoded ndarray with classification values

    config(dict): dictionary with model configuration
    

    Returns:

    depending on config, returns a dictionary with class weights or None

    '''
    if config['class_weights']:
        y_labels= np.argmax(y,axis=1)
        class_labels =np.unique(y_labels)
        class_weights =compute_class_weight('balanced',class_labels,y_labels)
        print('Class weights:')
        print(dict_weights)
        dict_weights= dict(enumerate(class_weights))
        return dict_weights
    else:
        return None

In [0]:
def  early_stopping(config):
    ''' Returns a encoder layer

    
    Parameters:

    config(dict): dictionary with model configuration

    
    Returns:

    depending on config, returns an instance of EarlyStopping or None

    '''
    if config['early_stopping']:
        return EarlyStopping(monitor= 'val_loss',
                             min_delta= config['early_stopping_delta'],
                             patience= config['early_stopping_patience'],
                             restore_best_weights=True)
    else:
        return None 

In [0]:
def encoder_conv_blocks(num_filters, en, config):
    ''' Returns a tensor  corresponding to a convolution block of an encoder 

    
    Parameters:

    num_filtes(int): number of filters for the convolution block

    en(4d tensor): tensor corresponding to previous block

    config(dict): dictionary with  model configuration

    
    Returns:

    en(4d tensor): tensor corresponding to a convolution block

    '''
    kernel_regularizer = assert_kernel_regularizer(config)
    activity_regularizer = assert_activity_regularizer(config)
    regularization_control = config['init_num_filters']*(2*config['conv_blocks'])

    for i in range(0,config['layers_per_block']):
        en= Conv2D(num_filters,
                 (3,3), 
                 padding='same', 
                 strides=(1,1),
                 activation='relu', 
                 kernel_initializer='glorot_uniform', 
                 kernel_regularizer= kernel_regularizer,
                 activity_regularizer= activity_regularizer)(en)
        if config['batch_norm']:
            en= BatchNormalization()(en) 
        if config['gaussian_noise_hidden'] and num_filters < regularization_control:
            en= GaussianNoise(config['gaussian_noise_stddev'])(en)
    en= MaxPooling2D((2,2))(en)
    if config['dropout'] and num_filters < regularization_control:
        en=Dropout(config['dropout_value'])(en)
    return en

In [0]:
def decoder_conv_blocks(num_filters, de, config):
    ''' Returns a tensor  corresponding to the convolution block of a decoder

    
    Parameters:

    num_filters(int): number of filters for the convolution block

    de(4d tensor): tensor corresponding to previous block

    config(dict): dictionary with  model configuration

    
    Returns:

    en(4d tensor): tensor correspondind to a convolution block

    '''
    for i in range(0,config['layers_per_block']):
        de= Conv2D(num_filters,
                  (3,3), 
                  padding='same', 
                  activation='relu',
                  strides=(1,1))(de)
        if config['batch_norm']:
            de= BatchNormalization()(de)
    de= UpSampling2D((2,2))(de)
    return de

In [0]:
def create_encoder(num_filters, en, config):
    ''' Returns a tensor  corresponing to an encoder (at each new conv block the 
    bumber of filters is multiplied by 2)

    Parameters:

    num_filtes(int): number of filters to start the encoder with

    en(4d tensor): tensor corresponding to previous block

    config(dict): dictionary with  model configuration

    
    Returns:

    en(4d tensor), num_filters(int): tensor corresponding to an encoder and number of 
    filters of its last convolution block 

    '''
    for i in range(0,config['conv_blocks']):
        if i==0:
            en= encoder_conv_blocks(num_filters, en, config)
        else:
            num_filters *= 2
            en= encoder_conv_blocks(num_filters, en, config)
    return en, num_filters

In [0]:
def create_decoder(num_filters, de, config):
    ''' Returns a tensor  corresponing to a decoder

    
    Parameters:

    num_filters(int): number of filters to start the decoder with

    de(4d tensor): tensor corresponding to previous block

    config(dict): dictionary with  model configuration

    
    Returns:

    de(4d tensor), num_filters(int): tensor corresponding to a decoder and number of 
    filters of its last convolution block 

    '''
    for i in range(0,config['conv_blocks']):
        if i==0:
            de= decoder_conv_blocks(num_filters, de, config)
        else:
            num_filters //= 2
            de= decoder_conv_blocks(num_filters, de, config)
    return de, num_filters

In [0]:
def create_autoencoder(config):
    ''' Returns an autoencoder model based on config

    
    Parameters

    config(dict): dictionary with  model configuration

    
    Returns:

    autoencoder(Model): Model instance of an autoencoder

    '''

    num_filters= config['init_num_filters']
    optimizer = assert_optimizer(config)
    img_input= Input(shape=config['image_shape'])

    en=img_input
    if config['gaussian_noise_input']:
        en=GaussianNoise(config['gaussian_noise_stddev'])(en)

    #Encoder  
    encoded, num_filters = create_encoder(num_filters, en, config)
    encoder= Model(img_input,encoded,name='encoder')
    encoder.summary()

    #Decoder
    encoded_input= Input(shape=encoder.output_shape[1:])
    de,_= create_decoder(num_filters, encoded_input,config)
    decoded= Conv2D(3,
                    (3,3), 
                    padding='same', 
                    activation='sigmoid',
                    kernel_initializer='glorot_uniform')(de)

    decoder= Model(encoded_input,decoded, name='decoder')
    decoder.summary()

    autoencoder= Model(img_input,decoder(encoder(img_input)))
    autoencoder.compile(optimizer, 
                    loss= config['loss'])
    autoencoder.summary()

    return autoencoder

In [0]:
def train_autoencoder(x_train, x_val, config, autoencoder_filename,encoder_filename):
    ''' Returns a trained autoencoder, saves trained autoencoder model, and 
    plots training and validation metrics

    
    Parameters

    x_train(4d array): 4d array with the training set

    x_val(4d array): 4d array array with the validation set

    config(dict): dictionary with  model configuration

    
    Returns:

    autoencoder(Model): instance of model with trained autoencoder

    '''

    if config['callbacks']:
        callbacks=[early_stopping(config)]
    else:
        callbacks=None

    autoencoder = create_autoencoder(config)
    autoencoder.summary()
    history= autoencoder.fit(x_train,
                            x_train,
                            batch_size= config['batch_size'],
                            epochs=config['epochs'],
                            callbacks=callbacks,
                            validation_data=(x_val,x_val))
    plot_model_metrics(history,autoencoder_filename,loss=True, acc=False)
    autoencoder.save('models/'+ autoencoder_filename + '.h5')
    autoencoder.get_layer(name='encoder').save('models/'+ encoder_filename + '.h5')
    return autoencoder

In [0]:
def create_classifier(autoencoder_config, classifier_config, 
                      encoder_filename, classifier_filename):
    ''' Returns a cassifier model based on autoencoder_config and 
    classifier_config


    Parameters:

    autencoder_config(dict): dictionary with autoencoder model configuration

    classifier_config(dict): dictionary with classifier model configuration

    encoder_filename(string): filename without extension of the encoder
    to be loaded from '.../models/' directory
    
    classifier_filename(string): filename without extension of the classifier
    to be saved in '.../models/' directory

    Returns:

    classifier(Model): classifier model with convolution layers corresponding
    to encoder, encoder layers frozen and weights loaded from the encoder 
    component of the autoencoder

    '''

    num_filters= autoencoder_config['init_num_filters']
    shape_conv = autoencoder_config['image_shape']
    optimizer = assert_optimizer(classifier_config)
    img_input= Input(shape=autoencoder_config['image_shape'])

    # Encoder
    en, num_filters = create_encoder(num_filters, img_input, autoencoder_config)

    # Classifier
    c= Conv2D(num_filters,(3,3),
              padding='same',
              kernel_initializer='glorot_uniform')(en)
    if classifier_config['batch_norm']:
        c= BatchNormalization()(c)
    if classifier_config['dropout']:
        c= Dropout(classifier_config['dropout_value'])(c)

    if classifier_config['global_pooling']=='max':
        c= GlobalMaxPooling2D()(c)
    
    if classifier_config['global_pooling']=='average':
        c= GobalAveragePooling2D()(c)

    if classifier_config['global_pooling']=='flatten':
        c= Flatten()(c)
        c= Dense(num_filters, 
                 kernel_initializer='glorot_uniform',
                 activation='relu')(c)
        if classifier_config['batch_norm']:
            c= BatchNormalization()(c)
                 
    if (num_filters //2) > 16 and classifier_config['dense']:
        c= Dense(num_filters //2, 
                 kernel_initializer='glorot_uniform',
                 activation='relu')(c)
        c= BatchNormalization()(c)

    output= Dense(10, 
             activation='softmax',
             kernel_initializer='glorot_uniform')(c)
    classifier = Model(img_input, output)
    encoder= load_model('models/'+ encoder_filename +'.h5')

    if autoencoder_config['gaussian_noise_input']:
        initial_layer=2
        subtracted_layers=1
    else: 
        initial_layer=1
        subtracted_layers=0
    
    num_layers= len(encoder.layers)
    
    # load weights into encoder layers
    for l1,l2 in zip(classifier.layers[1:num_layers-subtracted_layers],encoder.layers[initial_layer:num_layers]):
        l1.set_weights(l2.get_weights())

    del encoder
    gc.collect()

    for l in classifier.layers[:num_layers-subtracted_layers]:
        l.trainable=False

    classifier.compile(optimizer, 
                        loss= classifier_config['loss'], 
                        metrics= ['accuracy'],
                        weighted_metrics=classifier_config['weighted_metrics'])
    return classifier

In [0]:
def train_classifier(x_train, x_val, y_train, y_val, 
                     autoencoder_config, classifier_config, 
                     encoder_filename, classifier_filename):
    
    ''' Returns a trained cassifier model based on autoencoder_config and 
    classifier_config, saves trained classifier and plots training
    and validation metrics


    Parameters:

    x_train(4d array): 4d array with the training set

    x_val(4d array): 4d array with the validation set

    y_train(2d array): one-hot encoded 2d array with classification 
    values for the training set

    y_val(2d array): one-hot encoded numpy array with classification 
    values for the validation set

    autencoder_config(dict): dictionary with autoencoder model configuration

    classifier_config(dict): dictionary with classifier model configuration

    autoencoder_filename(string): filename without extension of the autoencoder
    to be loaded from '.../models/' directory
    
    classifier_filename(string): filename without extension of the classifier
    to be loaded in '.../models/' directory


    Returns:

    classifier(Model): instance of Model with trained classifier
    '''
    
    classifier= create_classifier(autoencoder_config, 
                                  classifier_config,
                                  encoder_filename, 
                                  classifier_filename)
    classifier.summary()

    if classifier_config['callbacks']:
        callbacks=[early_stopping(config)]
    else:
        callbacks=None

    class_weight = get_class_weights(y_train, classifier_config)

    if classifier_config['data_augmentation']:
        print('Using real-time data augmentation.')
        datagen = ImageDataGenerator(width_shift_range=0.1, 
                                     height_shift_range=0.1, 
                                     fill_mode='nearest',
                                     rotation_range=40,
                                     zoom_range=0.1,
                                     horizontal_flip=True)
        
        datagen.fit(x_train, seed=42)
        history=classifier.fit_generator(datagen.flow(x_train, 
                                                      y_train, 
                                                      batch_size=classifier_config['batch_size']),
                                         steps_per_epoch=x_train.shape[0] // classifier_config['batch_size'],
                                         epochs=classifier_config['epochs'],
                                         validation_data=(x_val, y_val),
                                         callbacks=callbacks,
                                         class_weight=class_weight)
        
        plot_model_metrics(history, classifier_filename, classifier_config)
        save_history(history.history, classifier_filename)
        classifier.save('models/'+ classifier_filename + '.h5')
    else:
        history= classifier.fit(x_train,
                       y_train,
                       batch_size= classifier_config['batch_size'],
                       epochs=classifier_config['epochs'],
                       callbacks=callbacks,
                       validation_data=(x_val,y_val),
                       class_weight=class_weight)
        
        plot_model_metrics(history,classifier_filename,classifier_config)
        save_history(history.history, classifier_filename)
        classifier.save('models/'+ classifier_filename + '.h5')
    
    return classifier

In [0]:
def classifier_predict_evaluate( x, y, classifier, class_names):
    ''' Returns classifier prediction for x, plots confusion matrix, and prints
    classification report

    
    Parameters:

    x(4d array):  numpy array with image samples

    y(2d array):  one-hot encoded 4d array with ground truth for the classificaiton 
    of images in x

    classifier(Model): instance of Model with classifier

    class_names(list): list of unique classes in the dataset under study 

    
    Returns:

    y_pred(numpy array): one-hot encoded numpy array with the prediction for the
    classification of images in x
    '''
    y_pred = classifier.predict(x)
    cm= confusion_matrix(y.argmax(axis=1), y_pred.argmax(axis=1))
    plot_confusion_matrix(cm, class_names)
    print('')
    cr= classification_report(y.argmax(axis=1),
                              y_pred.argmax(axis=1), 
                              target_names=class_names)
    print(cr)
    y_pred = np_utils.to_categorical(y_pred.argmax(axis=1))
    return y_pred

In [0]:
def plot_model_metrics(model_history, filename, loss=True, acc=True):
    ''' Plots and saves (in directory 'plots/') training and validation loss 
    and accuracy

    Parameters:

    model_history(keras History): keras History object with records of training and 
    validation loss and accuracy

    filename(string): name to be concatenated with loss and accuracy plot files 

    loss(boolean): whether to plot loss and save plot

    acc(boolean): whether to plot accuracy and save the plot

    '''
    if loss:
        #plot loss
        fig1= plt.figure(1)
        plt.plot(model_history.history['loss'])
        plt.plot(model_history.history['val_loss'])
        plt.title('Model Loss')
        plt.ylabel('loss')
        annotation1 = 'Train loss: ' + str(round(model_history.history['loss'][len(model_history.history['loss'])-1],4))
        annotation2 = 'Val loss: ' + str(round(model_history.history['val_loss'][len(model_history.history['val_loss'])-1],4))
        annotation = annotation1 + ', ' + annotation2
        plt.xlabel('epoch \n' + annotation)
        plt.legend(['train','validation'],loc='upper right')
        filepath = 'plots/' + filename + '_loss.png'
        plt.savefig(filepath, dpi=300,  bbox_inches = "tight")

    if acc:
        #plot accuracy
        fig2= plt.figure(2)
        plt.plot(model_history.history['acc'])
        plt.plot(model_history.history['val_acc'])
        plt.title('Model Accuracy')
        plt.ylabel('accuracy')
        annotation1 = 'Train accuracy: ' + str(round(model_history.history['acc'][len(model_history.history['acc'])-1],4))
        annotation2 = 'Val accuracy: ' + str(round(model_history.history['val_acc'][len(model_history.history['val_acc'])-1],4))
        annotation = annotation1 + ', ' + annotation2
        plt.xlabel('epoch \n' + annotation)
        plt.legend(['train','validation'],loc='upper right')
        filepath = 'plots/'+ filename + '_accuracy.png'
        plt.savefig(filepath,dpi=300, bbox_inches = "tight")

In [0]:
def plot_histories(history_list, caption_list, filename,train= False, metric=None):

    if train:
        title = 'Train'
        loss= 'loss'
        acc= 'acc'
        metric= metric
    else:
        title = 'Validation'
        loss= 'val_loss'
        acc= 'val_acc'
        metric= 'val_' + metric


    loaded_histories=[]
    for i in history_list:
        history=load_history(i)
        loaded_histories += history
    
    # loss
    fig1= plt.figure(1)
        
    for h in loaded_histories:
        plt.plot(h.history[loss])
    
    plt.title(title+'Loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(caption_list,loc='best')
    filepath = 'plots/' + filename + title+'_loss.png'
    plt.savefig(filepath, dpi=300,  bbox_inches = "tight")
      
    # accuracy
    fig2= plt.figure(2)

    for h in loaded_histories:
        plt.plot(h.history[acc])

    plt.title(title + 'Model Accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(caption_list,loc='best')
    filepath = 'plots/'+ filename + title + '_accuracy.png'
    plt.savefig(filepath,dpi=300, bbox_inches = "tight")

    if metric:
        fig3= plt.figure(3)

        for h in loaded_histories:
            plt.plot(h.history[metric])

        plt.title(title + 'Model' + metric)
        plt.ylabel(metric)
        plt.xlabel('epoch')
        plt.legend(caption_list,loc='best')
        filepath = 'plots/'+ filename + '_' + metric + '.png'
        plt.savefig(filepath,dpi=300, bbox_inches = "tight")

In [0]:
def reset_weights(model):
    
    session = K.get_session()
    for layer in model.layers: 
        if hasattr(layer, 'kernel_initializer'):
            layer.kernel.initializer.run(session=session)